In [1]:
import numpy as np
import torch # PyTorch package
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import *
from torchsummary import summary
from pytorchtools import EarlyStopping
import time
from sklearn.metrics import classification_report, confusion_matrix
import splitfolders
import os
import re
import PIL
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.use("Agg")
import warnings
warnings.filterwarnings('ignore')

#importing functions and parameters files
from functions import Net, ImageSequenceDataset, ImageSequenceClassifier
from params import bs, num_epochs, device, patience, lr

#### Let's split the training folder into training and validation folders.

In [2]:
#only run one time
#splitfolders.ratio("images/training", output="training-validation",
#    seed=1337, ratio=(.8, .2), group_prefix=None, move=False) #

In [3]:
transform = transforms.Compose([
    transforms.CenterCrop(1080),
    transforms.Resize(224),
    transforms.ToTensor()])

In [4]:
#loading the data
train_data = datasets.ImageFolder('training-validation/train', transform = transform)
val_data = datasets.ImageFolder('training-validation/val', transform = transform)
test_data = datasets.ImageFolder('images/testing', transform = transform)
seq_data = datasets.ImageFolder('images/training', transform = transform)
print(train_data.classes)

['flip', 'notflip']


In [5]:
# initialize the train, validation, and test data loaders
trainloader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = bs)
valloader = torch.utils.data.DataLoader(val_data, batch_size = bs)
testloader = torch.utils.data.DataLoader(test_data, batch_size = bs)

In [6]:
# calculate steps per epoch for training and validation set
trainSteps = len(trainloader.dataset) // bs
valSteps = len(valloader.dataset) // bs

In [7]:
#inputs, classes = next(iter(trainloader))
#print(f'Batch size: {inputs.shape[0]}')
#print(f'Number of color channels: {inputs.shape[1]}')
#print(f'Image size: {inputs.shape[2]} x {inputs.shape[3]}')

train_inputs, train_classes = next(iter(trainloader))
print(f'train input size: {train_inputs.shape}, train class size: {train_classes.shape}')
val_inputs, val_classes = next(iter(valloader))
print(f'val input size: {val_inputs.shape}, val class size: {val_classes.shape}')
test_inputs, test_classes = next(iter(testloader))
print(f'test input size: {test_inputs.shape}, test class size: {test_classes.shape}')

train input size: torch.Size([16, 3, 224, 224]), train class size: torch.Size([16])
val input size: torch.Size([16, 3, 224, 224]), val class size: torch.Size([16])
test input size: torch.Size([16, 3, 224, 224]), test class size: torch.Size([16])


In [8]:
# visualizing the images
plt.figure(figsize=(20, 10))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(train_inputs[i].permute(1, 2, 0))
    plt.title([train_data.classes[train_classes[i]]])

In [9]:
# measure how long training is going to take
print("[INFO] training the network...")
startTime = time.time()

[INFO] training the network...


In [10]:
#calling our model
model = Net()

# initialize our optimizer and loss function
# specify loss function
lossFn = nn.BCELoss() 
# specify optimizer
opt = optim.Adam(model.parameters(), lr=lr)

In [11]:
#Train the model using early stopping
def train_model(model, bs, patience, num_epochs):

	# to store training history
	train_loss = []
	train_acc = []
	val_loss = []
	val_acc = []

	# initialize the early_stopping object
	early_stopping = EarlyStopping(patience=patience, verbose=True)

	#training loop
	# loop over our epochs
	for e in range(0, num_epochs):
		# set the model in training mode
		model.train()
		# initialize the total training and validation loss
		totalTrainLoss = 0
		totalValLoss = 0
		# initialize the number of correct predictions in the training
		# and validation step
		trainCorrect = 0
		valCorrect = 0
		# loop over the training set
		for (x, y) in trainloader:
			# send the input to the device
			(x, y) = (x.to(device), y.to(device))
			# clear the gradients of all optimized variables
			opt.zero_grad()
			# perform a forward pass and calculate the training loss
			pred = model(x)
			loss = lossFn(pred, y.type(torch.float32).unsqueeze(1))
			# zero out the gradients, perform the backpropagation step,
			# and update the weights
			loss.backward()
			opt.step()
			# add the loss to the total training loss so far and
			# calculate the number of correct predictions
			totalTrainLoss += loss
			trainCorrect += (torch.round(pred) == y).type(torch.float).mean().item()

		# Zeroing gradient, performing backpropagation, updating weights of the model
		# set the model in evaluation mode
		with torch.no_grad():
			model.eval() # prep model for evaluation
			# loop over the validation set
			for (x, y) in valloader:
				# send the input to the device
				(x, y) = (x.to(device), y.to(device))
				# make the predictions and calculate the validation loss
				pred = model(x)
				totalValLoss += lossFn(pred, y.type(torch.float32).unsqueeze(1))
				# calculate the number of correct predictions
				valCorrect += (torch.round(pred) == y).type(torch.float).mean().item()
	
		# calculate the average training and validation loss
		avgTrainLoss = totalTrainLoss / trainSteps
		avgValLoss = totalValLoss / valSteps
		
		# calculate the training and validation accuracy
		avgtrainCorrect = trainCorrect / len(trainloader)
		avgvalCorrect = valCorrect / len(valloader)
		
		# update our training history
		train_loss.append(avgTrainLoss.cpu().detach().numpy())
		train_acc.append(avgtrainCorrect)
		val_loss.append(avgValLoss.cpu().detach().numpy())
		val_acc.append(avgvalCorrect)
		
		# print the model training and validation information
		print("[INFO] Epoch: {}/{}".format(e + 1, num_epochs))
		print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(avgTrainLoss, avgtrainCorrect))
		print("Valid loss: {:.6f}, Valid accuracy: {:.4f}\n".format(avgValLoss, avgvalCorrect))

		# early_stopping needs the validation loss to check if it has decresed, 
		# and if it has, it will make a checkpoint of the current model
		early_stopping(avgValLoss, model)
        
		if early_stopping.early_stop:
			print("Early stopping")
			break
	
	# load the last checkpoint with the best model
	model.load_state_dict(torch.load('checkpoint.pt'))
	
	return  model, train_loss, train_acc, val_loss, val_acc

In [12]:
#model, AvgTrainLoss, avgtrainCorrect, avgValLoss, avgvalCorrect = 
#train_model(model, bs, patience, num_epochs)

In [14]:
# visualize the loss as the network trained
#fig = plt.figure(figsize=(10,8))
#plt.plot(range(1,len(AvgTrainLoss)+1),AvgTrainLoss, label='Training Loss')
#plt.plot(range(1,len(avgValLoss)+1),avgValLoss,label='Validation Loss')

# find position of lowest validation loss
#minposs = avgValLoss.index(min(avgValLoss))+1 
#plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

#plt.xlabel('Epochs')
#plt.ylabel('Loss')
#plt.ylim(0, 0.5) # consistent scale
#plt.xlim(0, len(AvgTrainLoss)+1) # consistent scale
#plt.grid(True)
#plt.legend()
#plt.tight_layout()
#plt.show()
#fig.savefig('loss_plot.png', bbox_inches='tight')

In [15]:
# finish measuring how long training took
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))

[INFO] total time taken to train the model: 37.75s


In [16]:
def test_model(model, lossFn, testloader):
  model.eval()
  test_loss = 0
  correct = 0
  pred_list, true_list = [], []

  with torch.no_grad():
    for (x, y) in testloader:
      if len(x.shape) == 3:
        x = torch.unsqueeze(x,0)
      out = model(x).flatten()
      if type(x)!=type(y):
        y=torch.Tensor([y])
      test_loss += lossFn(out, y.type(torch.float32))
      correct += torch.round(out).eq(y).sum()
      pred_list.append(torch.round(out))
      true_list.append(y.type(torch.float32))

      # Print every 100 iterations
      if (i + 1) % 100 == 0:
        print(f"Iteration {i+1}/{len(testloader)}")
      
    test_loss /= len(testloader)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(testloader),
            100. * correct / len(testloader)))
    
  return pred_list, true_list

#predictions, labels = test_model(model, lossFn, test_data)

In [18]:
# generate a classification report
#print(classification_report([i.item() for i in predictions], [i.item() for i in labels]))

#### Classifying Sequence of Images

In [19]:
#To write a list of file names

#Define the path to the folder
#folder_path = "images/training"

#Function to get file names with a specific extension from a folder
#def get_file_names(folder_path):
#    file_names = []
#    for item in os.listdir(folder_path):
#        item_path = os.path.join(folder_path, item)
#        if os.path.isfile(item_path):
#            file_names.append(item)
#    return file_names

# Get file names from the folder
#file_names = get_file_names(folder_path)

#print("File names:", file_names)

In [20]:
#variable sequence size

# Extract sequence size from file names first 4 characters
#sequence_size = len(re.findall(r'^\d{4}', file_names[0]))

#print("Sequence size:", sequence_size)

In [32]:
import torchvision.models as models
sequence_length = 5

# Create a DataLoader
seqloader = DataLoader(seq_data, batch_size= bs*sequence_length , shuffle=True)

# Create an instance of the model
num_classes = 2 
hidden_size = 128 # Adjust 
model = ImageSequenceClassifier(num_classes, hidden_size)

# Iterate through the dataloader and pass sequences to the model
for epoch in range(num_epochs):
    for images, labels in seqloader:
        opt.zero_grad()
        outputs = model(images.unsqueeze(1))
        loss = lossFn(outputs, labels.unsqueeze(1).type(torch.float32))
        loss.backward()
        opt.step()

ValueError: Using a target size (torch.Size([80, 1])) that is different to the input size (torch.Size([80, 2])) is deprecated. Please ensure they have the same size.